# Prepear test dataset
Format - {word}{several spaces}{answer-words separeted by comma}

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pandas
import pandas as pd

In [3]:
test_incorrect_file = open('../data/test_sample_incorrect.txt', 'r', encoding="utf8")
test_incorrect_lines = test_incorrect_file.readlines()
splitted_test_incorrect_lines = list(map(lambda x: x.split(maxsplit=1), test_incorrect_lines))
test_incorrect_words = list(map(lambda x: x[0], splitted_test_incorrect_lines))
test_incorrect_answers = list(map(lambda x: list(map(lambda y: y.strip(), x[1].split(', '))), splitted_test_incorrect_lines))
test_df = pd.DataFrame(test_incorrect_words, columns=["test_word"])
test_df["answers"] = test_incorrect_answers


In [4]:
test_df.head()

,test_word,answers
0,эктренном,[экстренном]
1,синусовго,[синусового]
2,даным,"[данным, данный, данные]"
3,шнутография,[шунтография]
4,сникопальных,[синкопальных]


In [5]:
test_word_list = test_df["test_word"].tolist()

In [6]:
test_answers = test_df["answers"].tolist()

In [7]:
# Need to create test example for computing lexical precision
def compute_error_precision(original_word_list, corrected_word_list, answer_word_list):
    words_number = len(corrected_word_list)
    correct_words_number = 0
    print("original_word_list --- corrected_word --- answer_word_list")
    for i, corrected_word in enumerate(corrected_word_list):
        print(f"{original_word_list[i]} --- {corrected_word} --- {answer_word_list[i]}")
        if corrected_word in answer_word_list[i]:
            correct_words_number += 1
    print(f"Right corrected words count - {correct_words_number} of {words_number} total")
    return correct_words_number/words_number

In [8]:
from tqdm import tqdm

In [9]:
def compute_all_metrics(spellchecker_function, original_word_list, answer_word_list):
    result = spellchecker_function()
    elapsed_time = result["elapsed"]
    corrected_word_list = result["corrected_word_list"]
    error_precision = compute_error_precision(original_word_list, corrected_word_list, answer_word_list)
    # TODO: Compute lexical precision
    lexical_precision = 0.0
    return elapsed_time / len(original_word_list), \
           error_precision, \
           lexical_precision, \
           original_word_list, \
           corrected_word_list,\
           answer_word_list

# Spellchecker prototype initialization

In [10]:
!pip install pymorphy2 
!pip install gensim
!pip install python-levenshtein
!pip install stringdist
!pip install tqdm

# Test SpellChecker Prototype v.2

In [ ]:
from spellchecker_prototype_v2.spell_checker import SpellChecker

KeyboardInterrupt: 

In [2]:
def spellchecker_prototype_v2_correct_func():
    spellchecker_prototype = SpellChecker()
    input_text_for_prototype = " ".join(test_word_list)
    timer = tqdm()
    corrected_word_list = spellchecker_prototype.correct_words(input_text_for_prototype)
    return {"elapsed": timer.format_dict["elapsed"], "corrected_word_list": corrected_word_list}

spellchecker_prototype_v2_metrics = compute_all_metrics(spellchecker_prototype_v2_correct_func, test_word_list, test_answers)
spellchecker_prototype_v2_metrics

NameError: name 'compute_all_metrics' is not defined

# Test Symspellpy - https://github.com/mammothb/symspellpy

In [ ]:
!pip install -U symspellpy

### Test Symspellpy lookup on basic dict

In [11]:
basic_frequency_dict = '../data/symspell/ru-100k.txt'

In [12]:
from symspellpy import SymSpell, Verbosity

def test_symspell_py_lookup(frequency_dict_path, input_word_list):
    sym_spell_py = SymSpell()
    sym_spell_py.load_dictionary(frequency_dict_path, 0, 1, encoding="UTF8")

    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = sym_spell_py.lookup(word, Verbosity.TOP, max_edit_distance=2, include_unknown=True)
        result.append(suggestions[0].term)
    return  {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}

def test_symspell_py_lookup_compound(frequency_dict_path, input_word_list):
    sym_spell_py = SymSpell()
    sym_spell_py.load_dictionary(frequency_dict_path, 0, 1, encoding="UTF8")

    result = []
    timer = tqdm(input_word_list)
    for word in timer:
        suggestions = sym_spell_py.lookup_compound(word, max_edit_distance=2)
        result.append(suggestions[0].term)
    return {"elapsed" : timer.format_dict["elapsed"], "corrected_word_list" : result}



In [13]:
test_symspell_py_lookup_result = compute_all_metrics(
    lambda : test_symspell_py_lookup(basic_frequency_dict, test_word_list), test_word_list, test_answers)
test_symspell_py_lookup_result

100%|██████████| 200/200 [00:00<00:00, 7143.31it/s]

original_word_list --- corrected_word --- answer_word_list
эктренном --- коренном --- ['экстренном']
синусовго --- синусовго --- ['синусового']
даным --- данным --- ['данным', 'данный', 'данные']
шнутография --- шнутография --- ['шунтография']
сникопальных --- сникопальных --- ['синкопальных']
посление --- последние --- ['поселение', 'последний']
ходавый --- подавай --- ['ходовый']
анмнезе --- анамнезе --- ['анамнезе', 'анамнез']
сентированный --- сентированный --- ['стентированный']
самоч --- самой --- ['самочувствие']
перевдена --- переведена --- ['переведена']
стентированеим --- стентированеим --- ['стентированием', 'стентирование']
кардиол --- кардинал --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- догосптальном --- ['догоспитальном']
миним --- синим --- ['минимум', 'минимальный']
субфебриллитет --- субфебриллитет --- ['субфебрилитет']
загрудиные --- загрудиные --- ['загрудинные', 'загрудинный']
перферия --- периферия --- ['перифери

(0.00014653325080871582,
 0.42,
 0.0,
 ['эктренном',
  'синусовго',
  'даным',
  'шнутография',
  'сникопальных',
  'посление',
  'ходавый',
  'анмнезе',
  'сентированный',
  'самоч',
  'перевдена',
  'стентированеим',
  'кардиол',
  'переведна',
  'догосптальном',
  'миним',
  'субфебриллитет',
  'загрудиные',
  'перферия',
  'течени',
  'оклюзия',
  'жнщина',
  'пциент',
  'зарегистирована',
  'сегмета',
  'клекте',
  'эпид',
  'интелектуально',
  'нижн',
  'прводилась',
  'неуст',
  'госптализирован',
  'вперые',
  'послеоперационом',
  'госпитализирова',
  'ухудщение',
  'рентг',
  'дальнешем',
  'троботическая',
  'многосоудистое',
  'миокрада',
  'сопрвождались',
  'дискмфорт',
  'потлтвостью',
  'левогожелудочка',
  'рание',
  'виед',
  'ангинозныеболи',
  'догоситальном',
  'ввполнен',
  'чаосв',
  'октазалась',
  'линика',
  'ммента',
  'проведеной',
  'выполненяи',
  'леченяи',
  'обтилась',
  'повтороно',
  'зарегистриованы',
  'отицает',
  'дальнейщем',
  'болисохраняются',

In [14]:
test_symspell_py_lookup_compound_result = compute_all_metrics(
    lambda : test_symspell_py_lookup_compound(basic_frequency_dict, test_word_list), test_word_list, test_answers)
test_symspell_py_lookup_compound_result

100%|██████████| 200/200 [00:00<00:00, 570.50it/s]


original_word_list --- corrected_word --- answer_word_list
эктренном --- коренном --- ['экстренном']
синусовго --- синус его --- ['синусового']
даным --- данным --- ['данным', 'данный', 'данные']
шнутография --- и фотография --- ['шунтография']
сникопальных --- они локальных --- ['синкопальных']
посление --- последние --- ['поселение', 'последний']
ходавый --- ход вый --- ['ходовый']
анмнезе --- анамнезе --- ['анамнезе', 'анамнез']
сентированный --- центров данный --- ['стентированный']
самоч --- самой --- ['самочувствие']
перевдена --- переведена --- ['переведена']
стентированеим --- стен титрованием --- ['стентированием', 'стентирование']
кардиол --- кардинал --- ['кардиолог,кардиологический']
переведна --- переведена --- ['переведена']
догосптальном --- до остальном --- ['догоспитальном']
миним --- синим --- ['минимум', 'минимальный']
субфебриллитет --- себе бриллиант --- ['субфебрилитет']
загрудиные --- за рудные --- ['загрудинные', 'загрудинный']
перферия --- периферия --- ['периф

(0.001753671169281006,
 0.46,
 0.0,
 ['эктренном',
  'синусовго',
  'даным',
  'шнутография',
  'сникопальных',
  'посление',
  'ходавый',
  'анмнезе',
  'сентированный',
  'самоч',
  'перевдена',
  'стентированеим',
  'кардиол',
  'переведна',
  'догосптальном',
  'миним',
  'субфебриллитет',
  'загрудиные',
  'перферия',
  'течени',
  'оклюзия',
  'жнщина',
  'пциент',
  'зарегистирована',
  'сегмета',
  'клекте',
  'эпид',
  'интелектуально',
  'нижн',
  'прводилась',
  'неуст',
  'госптализирован',
  'вперые',
  'послеоперационом',
  'госпитализирова',
  'ухудщение',
  'рентг',
  'дальнешем',
  'троботическая',
  'многосоудистое',
  'миокрада',
  'сопрвождались',
  'дискмфорт',
  'потлтвостью',
  'левогожелудочка',
  'рание',
  'виед',
  'ангинозныеболи',
  'догоситальном',
  'ввполнен',
  'чаосв',
  'октазалась',
  'линика',
  'ммента',
  'проведеной',
  'выполненяи',
  'леченяи',
  'обтилась',
  'повтороно',
  'зарегистриованы',
  'отицает',
  'дальнейщем',
  'болисохраняются',
 

# Test SymSpellCppPy - https://github.com/viig99/SymSpellCppPy

In [27]:
!pip install wheel setuptools

In [29]:
!pip install --upgrade SymSpellCppPy

  Using cached SymSpellCppPy-0.0.14.tar.gz (5.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for SymSpellCppPy
Failed to build SymSpellCppPy
    Running setup.py install for SymSpellCppPy: started
    Running setup.py install for SymSpellCppPy: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'c:\users\pogre\appdata\local\programs\python\python39\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\pogre\\AppData\\Local\\Temp\\pip-install-fkh0s3wn\\symspellcpppy_2da100226ca64a51aded2561628e791d\\setup.py'"'"'; __file__='"'"'C:\\Users\\pogre\\AppData\\Local\\Temp\\pip-install-fkh0s3wn\\symspellcpppy_2da100226ca64a51aded2561628e791d\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\pogre\AppData\Local\Temp\pip-wheel-0o0_me56'
       cwd: C:\Users\pogre\AppData\Local\Temp\pip-install-fkh0s3wn\symspellcpppy_2da100226ca64a51aded2561628e791d\
  Complete output (45 lines):
  running bdist_wheel
  running build
  running build_ext
  Traceba

There was a problem with installing the package. It is still unclear what the reason is. However, this spellchecker should not provide more accuracy, only higher speed since it is implemented in C++. However, the speed of the Python implementation is still enough at this stage.  So I decided to leave it here for now and come back if there is a need for more speed.

# Test JunSpell - https://github.com/bakwc/JamSpell

#### Attention No Windows 10 support!

In [31]:
!sudo apt-get install swig

"sudo" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [30]:
!pip install jamspell

  ERROR: Command errored out with exit status 1:
   command: 'c:\users\pogre\appdata\local\programs\python\python39\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\pogre\\AppData\\Local\\Temp\\pip-install-gtwd7y3t\\jamspell_18ec665c3f74451087bf50fc7ac9509f\\setup.py'"'"'; __file__='"'"'C:\\Users\\pogre\\AppData\\Local\\Temp\\pip-install-gtwd7y3t\\jamspell_18ec665c3f74451087bf50fc7ac9509f\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\pogre\AppData\Local\Temp\pip-wheel-5mzpgtdg'
       cwd: C:\Users\pogre\AppData\Local\Temp\pip-install-gtwd7y3t\jamspell_18ec665c3f74451087bf50fc7ac9509f\
  Complete output (41 lines):
  running bdist_wheel
  running build
  running build_ext
  building '_jamspell' e


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for jamspell
Failed to build jamspell
    Running setup.py install for jamspell: started
    Running setup.py install for jamspell: finished with status 'error'


        self.build_extensions()
      File "c:\users\pogre\appdata\local\programs\python\python39\lib\distutils\command\build_ext.py", line 449, in build_extensions
        self._build_extensions_serial()
      File "c:\users\pogre\appdata\local\programs\python\python39\lib\distutils\command\build_ext.py", line 474, in _build_extensions_serial
        self.build_extension(ext)
      File "c:\users\pogre\appdata\local\programs\python\python39\lib\distutils\command\build_ext.py", line 507, in build_extension
        sources = self.swig_sources(sources, ext)
      File "c:\users\pogre\appdata\local\programs\python\python39\lib\distutils\command\build_ext.py", line 598, in swig_sources
        swig = self.swig or self.find_swig()
      File "C:\Users\pogre\AppData\Local\Temp\pip-install-gtwd7y3t\jamspell_18ec665c3f74451087bf50fc7ac9509f\setup.py", line 49, in find_swig
        assert swigBinary is not None
    AssertionError
    ----------------------------------------
ERROR: Command error